# Deploying TAS4K8s to TKG standalone on vSphere 6.7U3

* TKG 1.1.2
* No load balancer
* Uses vsphere CNS through CSI

In [ ]:
#env
export TAS_ROOT=~/Downloads/tas-k8s-beta03-workshop-tkg/tanzu-application-service
export TAS_CONFIG_DIR=~/Downloads/tas-k8s-beta03-workshop-tkg/configuration-values
export TAS_ARCHIVE=~/Downloads/tanzu-application-service.0.3.0-build.114.tar
mkdir -p $TAS_ROOT $TAS_CONFIG_DIR
cd $TAS_ROOT

In [ ]:
#target the right cluster
kubectl config get-contexts

In [ ]:
kubectl config use-context tas-cluster-admin@tas-cluster

In [ ]:
kubectl get all -o wide --all-namespaces

# Deploy TAS

In [ ]:
#extract TAS downloaded archive. 
tar -C $TAS_ROOT/../ -xvf $TAS_ARCHIVE 

In [ ]:
#remove resource requirements in case you have less than 5 worker nodes wtih 8GB RAM each
curl https://raw.githubusercontent.com/cloudfoundry/cf-for-k8s/master/config-optional/remove-resource-requirements.yml > $TAS_ROOT/config-optional/remove-resource-requirements.yml

## Generate config values

In [ ]:
./bin/generate-values.sh -d "tas-tkg.lan" > ../configuration-values/deployment-values.yml

Add entries for Pivnet registry and your own image registry (place at end of file, or in two separate files, system-registry-values.yml and app-registry-values.yml):
```
system_registry:
  hostname: registry.pivotal.io
  username: pivnet-username
  password: password

app_registry:
  hostname: https://index.docker.io/v1/
  repository: "username"
  username: "uername"
  password: "password"
```

In [ ]:
#Copy my secrets from pre-existing location
cp ~/Documents/app-registry-values.yml $TAS_CONFIG_DIR
cp ~/Documents/system-registry-values.yml $TAS_CONFIG_DIR

## Default storage class

In [ ]:
kubectl get storageclass

Follow the instructions to configure storage class for TKG
https://docs.pivotal.io/tas-kubernetes/0-3/configuring-default-storage.html


In [ ]:
#for TKG on vSphere, take note of docs above to change parameters in below yaml for default storage class to target a datastore
kapp deploy -a default-storage-class -f config-optional/vsphere-csi-storage-class.yaml -y

In [ ]:
#deploy metrics server
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.3.6/components.yaml

In [ ]:
#deploy tas
./bin/install-tas.sh ../configuration-values

### Configure DNS to external IPs of worker nodes

In [ ]:
kubectl get nodes --output='wide'

# Tear down environment

In [ ]:
#remove cf app from cluster
kapp delete -a cf -y